In [ ]:
from pathlib import Path
import pandas as pd


ripp_df = pd.read_csv('data/11379_corepeptides_scored_cAMP20250915-forRevision.tsv', sep='\t')
len_before = len(ripp_df)
ripp_df.drop_duplicates(subset=['corepeptide'], inplace=True)
# ripp_df = ripp_df.head(10)
print(f'Dropped {len_before - len(ripp_df)}, from {len_before} to {len(ripp_df)}')
# Output file
out_fasta = Path("ripp.fa")

with open(out_fasta, "w") as f:
    for _, row in ripp_df.iterrows():
        name = str(row['pep_id']).strip()
        seq = str(row['corepeptide']).strip().replace(" ", "")
        f.write(f">{name}\n{seq}\n")

print(f"FASTA saved to {out_fasta}")


import subprocess
import pandas as pd
from pathlib import Path
from Bio import SeqIO
from Bio import pairwise2
from tqdm import tqdm
from multiprocessing import Pool, cpu_count

# --- User inputs ---
query_fasta = "ripp.fa"         # query sequences
db_fasta    = "AMP_all.fa"   # subject database (needed for NW)
db_prefix   = "AMP_all"         # diamond db prefix
out_tsv     = "diamond_results.tsv"
out_csv     = "best_hits.csv"

# Full path to diamond binary
diamond_bin = str(Path.home() / "diamond")

# Choose algorithm
run_NW = True   # True = run Needleman-Wunsch, False = run DIAMOND

# --- Helper for NW per query ---
def nw_best_hit(args):
    q_record, subjects = args
    best_identity = 0
    for s in subjects:
        aln_score = pairwise2.align.globalxx(q_record.seq, s.seq, score_only=True)
        # normalize by full sequence length (max length of query or subject)
        aln_length = max(len(q_record.seq), len(s.seq))
        if aln_length == 0:
            continue
        identity = 100.0 * aln_score / aln_length

        if identity > best_identity:
            best_identity = identity
    return {"query_id": q_record.id, "similarity_score": best_identity}



# --- Needleman-Wunsch function ---
def run_needleman_wunsch(query_fasta, db_fasta):
    queries = list(SeqIO.parse(query_fasta, "fasta"))
    subjects = list(SeqIO.parse(db_fasta, "fasta"))

    with Pool(processes=cpu_count()) as pool:
        results = list(tqdm(pool.imap(nw_best_hit, [(q, subjects) for q in queries]),
                            total=len(queries),
                            desc="Running Needleman-Wunsch",
                            unit="seq"))

    return pd.DataFrame(results)

# --- DIAMOND blastp ---
def run_diamond(query_fasta, db_prefix, out_tsv):
    subprocess.run([
        diamond_bin, "blastp",
        "--query", query_fasta,
        "--db", db_prefix,
        "--out", out_tsv,
        "--outfmt", "6", "qseqid", "sseqid", "pident", "bitscore",
        "--evalue", "100",
        "--max-target-seqs", "1",
        "--threads", "1"
    ], check=True)

    cols = ["qseqid", "sseqid", "pident", "bitscore"]
    if Path(out_tsv).stat().st_size > 0:
        df = pd.read_csv(out_tsv, sep="\t", header=None, names=cols)
        best_hits = df.loc[df.groupby("qseqid")["bitscore"].idxmax()]
        best_hits = best_hits[["qseqid", "pident"]].copy()
        best_hits.columns = ["query_id", "similarity_score"]
    else:
        best_hits = pd.DataFrame(columns=["query_id", "similarity_score"])

    return best_hits

# --- Main ---
if run_NW:
    best_hits = run_needleman_wunsch(query_fasta, db_fasta)
else:
    best_hits = run_diamond(query_fasta, db_prefix, out_tsv)

# --- Ensure all queries are present ---
all_queries = [record.id for record in SeqIO.parse(query_fasta, "fasta")]
all_df = pd.DataFrame({"query_id": all_queries})
final = all_df.merge(best_hits, on="query_id", how="left").fillna(0)

# --- Save to CSV ---
final.to_csv(out_csv, index=False)
print(f"Done! Best hit similarity scores saved in {out_csv}")

# Visualize results
import matplotlib.pyplot as plt
import pandas as pd

# Assume your dataframe is in "final"
df = final.copy()
df["similarity_score"] = pd.to_numeric(df["similarity_score"], errors="coerce").clip(0, 100)

# Separate zero and nonzero
zero_count = (df["similarity_score"] == 0).sum()
nonzero = df.loc[df["similarity_score"] > 0, "similarity_score"]
total = len(df)

# Calculate percentages
perc_zero = 100 * zero_count / total if total > 0 else 0
perc_nonzero = 100 * (total - zero_count) / total if total > 0 else 0


In [ ]:
import matplotlib.pyplot as plt

# Use your dataframe directly (replace "final" with your df variable)
df = final.copy()

# Ensure similarity_score is numeric and within [0, 100]
df["similarity_score"] = pd.to_numeric(df["similarity_score"], errors="coerce").clip(0, 100)

# Create histogram
plt.figure(figsize=(6, 4.5))
plt.hist(
    df["similarity_score"],
    bins=200,
    range=(0, 100),
    edgecolor="black",
    linewidth=0.6
)

# Professional styling
plt.xlabel("Similarity score (%)", fontsize=14)
plt.ylabel("Number of peptides", fontsize=14)
plt.xticks(fontsize=12)
plt.yticks(fontsize=12)
plt.tight_layout()

# Save high-resolution publication-quality output
plt.savefig("similarity_score_histogram.png", dpi=200)
plt.savefig("similarity_score_histogram.pdf", dpi=200)

plt.show()
